In [1]:
%pip install -r requirements.txt

In [ ]:
import sys
sys.path.append('../..')

In [8]:
import os
import importlib
import numpy as np
import pandas as pd

from datetime import datetime
from warnings import filterwarnings
from pandas.tseries.offsets import MonthEnd

pd.set_option('display.max.columns', 300)
filterwarnings('ignore')

In [ ]:
from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig, TrainingManager, ForecastConfig, ForecastEngine

from core.calculator.deposits import DepositsCalculationType, DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels

### Подготовка сценария

In [9]:
# Дата из который мы прогнозируем
train_end = datetime(year=2024, month=9, day=30)

# Горизонт прогноза в месяцах
horizon = 3

In [10]:
# Данные для прогноза
scenario_data = {
    # Ожидаемый баланс на первую дату прогноза, задавать необязательно
     'expected_amount':      [np.nan for h in range(horizon)],
    # ССВ
     'SSV':                  [0.48 for h in range(horizon)],
    # ФОР
     'FOR':                  [4.5 for h in range(horizon)],
    # Трансфертные ставки
     'VTB_ftp_rate_[90d]':   [12.3 for h in range(horizon)],
     'VTB_ftp_rate_[180d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[365d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[548d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[730d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[1095d]': [12 for h in range(horizon)],
    
    # тестируем бакеты баланса
    'VTB_rate_[mass]_[0_500k)': [12 for h in range(horizon)],
    'VTB_rate_[mass]_[500k_1500k)': [12.1 for h in range(horizon)],
    'VTB_rate_[mass]_[1500k_5000k)': [12.2 for h in range(horizon)],
    'VTB_rate_[mass]_[5000k_15000k)': [12.3 for h in range(horizon)],
    'VTB_rate_[mass]_[15000k_inf)': [12.4 for h in range(horizon)],


    'VTB_rate_[priv]_[0_500k)': [12 for h in range(horizon)],
    'VTB_rate_[priv]_[500k_1500k)': [12.1 for h in range(horizon)],
    'VTB_rate_[priv]_[1500k_5000k)': [12.2 for h in range(horizon)],
    'VTB_rate_[priv]_[5000k_15000k)': [12.3 for h in range(horizon)],
    'VTB_rate_[priv]_[15000k_inf)': [12.4 for h in range(horizon)],


    'VTB_rate_[vip]_[0_15kk)': [12.1 for h in range(horizon)],
    'VTB_rate_[vip]_[15kk_30kk)': [12.2 for h in range(horizon)],
    'VTB_rate_[vip]_[30kk_50kk)': [12.3 for h in range(horizon)],
    'VTB_rate_[vip]_[50kk_100kk)': [12.4 for h in range(horizon)],
    'VTB_rate_[vip]_[100kk_200kk)': [12.5 for h in range(horizon)],
    'VTB_rate_[vip]_[200kk_300kk)': [12.6 for h in range(horizon)],
    'VTB_rate_[vip]_[300kk_500kk)': [12.7 for h in range(horizon)],
    'VTB_rate_[vip]_[500kk_inf)': [12.8 for h in range(horizon)],
    
    
    # Маржа бизнеса по срочностям
     'margin_[90d]':         [0.1 for h in range(horizon)],
     'margin_[180d]':        [0.1 for h in range(horizon)],
     'margin_[365d]':        [0.1 for h in range(horizon)],
     'margin_[548d]':        [0.1 for h in range(horizon)],
     'margin_[730d]':        [0.2 for h in range(horizon)],
     'margin_[1095d]':       [0.2 for h in range(horizon)],
    
    # Спред Привилегия - Массовый (на сколько в среднем ставки по сегменту Привилегия больше чем ставки по массовому сегменту)
     'priv_spread':          [0.4 for h in range(horizon)],
    # Спред ВИП - Массовый (на сколько в среднем ставки по сегменту ВИП больше чем ставки по массовому сегменту)
     'vip_spread':           [0.8 for h in range(horizon)],
    
    # Ниже три спреда по разным типам опциональности по отношению к безопциональным вкладам (Подразумевается, что они, как правило, отрицательные)
    # r - возможности пополнения, s - возможность расходных операций
    
    # На сколько ставка по расходным вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r0s1_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s0_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по расходно-пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s1_spread':          [-1.2 for h in range(horizon)],
    
    # Ставка по лучшему предложению сбера
     'SBER_max_rate':        [11.2 for h in range(horizon)],
    
    # Базовая ставка по НС
     'SA_rate':              [5 for h in range(horizon)]
}
scenario_df_user = pd.DataFrame(scenario_data)

In [11]:
scenario_df = preprocess_scenario(scenario_df_user, train_end, horizon)

## Парсим сценарии

In [143]:
def parse_scenario(scenario, scenario_df, sber_rate, sa_rate):
    """
    Парсим все, НС и макс ставку Сбера задаем сами
    scenario - спарсенные данные 
    scenario_df -  датафрейм с индексами и колонками
    
    """
    
    # FTP ставки
    ftp_rates_list = ['VTB_ftp_rate_[90d]', 
                  'VTB_ftp_rate_[180d]', 
                  'VTB_ftp_rate_[365d]',
                  'VTB_ftp_rate_[548d]', 
                  'VTB_ftp_rate_[730d]', 
                  'VTB_ftp_rate_[1095d]']
    
    
    # доли по балансам
    balance_buckets = {
    'mass': [['VTB_rate_[mass]_[0_500k)', 
             'VTB_rate_[mass]_[500k_1500k)', 
             'VTB_rate_[mass]_[1500k_5000k)', 
             'VTB_rate_[mass]_[5000k_15000k)', 
             'VTB_rate_[mass]_[15000k_inf)'], 68, 73, 6],
    
    'priv': [['VTB_rate_[priv]_[0_500k)',
            'VTB_rate_[priv]_[500k_1500k)',
            'VTB_rate_[priv]_[1500k_5000k)',
            'VTB_rate_[priv]_[5000k_15000k)',
            'VTB_rate_[priv]_[15000k_inf)'], 46, 51, 6],
    
    'vip': [['VTB_rate_[vip]_[0_15kk)',
            'VTB_rate_[vip]_[15kk_30kk)',
            'VTB_rate_[vip]_[30kk_50kk)',
            'VTB_rate_[vip]_[50kk_100kk)',
            'VTB_rate_[vip]_[100kk_200kk)',
            'VTB_rate_[vip]_[200kk_300kk)',
            'VTB_rate_[vip]_[300kk_500kk)',
            'VTB_rate_[vip]_[500kk_inf)'], 23, 28, 6]
    }
    
    
    
    scenario_res = pd.DataFrame(columns=scenario_df.columns)
    
    for i, ftp_name in enumerate(ftp_rates_list):

        scenario_res.loc[0, ftp_name] = scenario.iloc[14, 4+i]*100
        
        
        
    segm_dict = {'mass': 70,
                'priv': 48,
                'vip': 27}
    
    def get_segm_values(df, segm_dict): 
        for segm in segm_dict.keys():

            row = segm_dict[segm]

            # без опций
            list_rates = list_rates_gen(segm, '0', '0')
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = scenario.iloc[row, 4+i] * 100

            # с пополнением
            list_rates = list_rates_gen(segm, '1', '0')
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = scenario.iloc[row, 20+i] * 100

            # с пополнением и снятием
            list_rates = list_rates_gen(segm, '1', '1')
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = scenario.iloc[row, 28+i] * 100

            # со снятием
            list_rates = list_rates_gen(segm, '0', '1')
            # заполняем заглушками
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = 0.001

        return df
    
    
    def list_rates_gen(segm, repl, sub):

        list_rates = [f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[90d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[180d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[365d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[548d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[730d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[1095d]']

        return list_rates
    
    
    def get_balance_buckets_rates(df, balance_buckets):
        """
        df: то что имеем, scenario_res  
        scenario - считанный, уже есть в считанном
        """
        
        
        for segm in balance_buckets.keys():
            
            col_names = balance_buckets[segm][0]
            first_ind = balance_buckets[segm][1]
            last_ind = balance_buckets[segm][2]
            col_ind = balance_buckets[segm][3]
            
            if segm in ['mass', 'priv']:
                


                df.loc[0, col_names] = (scenario.iloc[first_ind:last_ind, col_ind]*100).values
                
            if segm in ['vip']:
                
                df.loc[0, col_names[:-1]] = (scenario.iloc[first_ind, col_ind]*100)
                df.loc[0, col_names[-1]] = (scenario.iloc[last_ind-1, col_ind]*100)
        
        return df
    
    scenario_res = get_segm_values(scenario_res, segm_dict)
    scenario_res = get_balance_buckets_rates(scenario_res, balance_buckets)
    
    scenario_res['SBER_max_rate'] = sber_rate
    scenario_res['rate_sa_weighted'] = sa_rate
    
    
    scenario_res = scenario_res.append([scenario_res]*2, ignore_index=True)
    scenario_res.index = scenario_df.index
    
    
    return scenario_res

In [42]:
# считанный сценарий
sc_v1_r20 = pd.read_excel('scenarios_retail/retail_scenario_v1.xlsx', sheet_name='Сценарий КС = 20%')
sc_v1_r21 = pd.read_excel('scenarios_retail/retail_scenario_v1.xlsx', sheet_name='Сценарий КС = 21%')
sc_v1_r22 = pd.read_excel('scenarios_retail/retail_scenario_v1.xlsx', sheet_name='Сценарий КС = 22%')

In [43]:
# считанный сценарий
sc_v2_r21 = pd.read_excel('scenarios_retail/retail_scenario_v2.xlsx', sheet_name='Сценарий КС = 21%')
sc_v2_r22 = pd.read_excel('scenarios_retail/retail_scenario_v2.xlsx', sheet_name='Сценарий КС = 22%')

In [151]:
parse_sc_v1_r20 = parse_scenario(sc_v1_r20, scenario_df, sber_rate=20, sa_rate=16)
parse_sc_v1_r21 = parse_scenario(sc_v1_r21, scenario_df, sber_rate=21, sa_rate=17)
parse_sc_v1_r22 = parse_scenario(sc_v1_r22, scenario_df, sber_rate=22, sa_rate=17.5)


parse_sc_v2_r21 = parse_scenario(sc_v2_r21, scenario_df, sber_rate=21, sa_rate=17)
parse_sc_v2_r22 = parse_scenario(sc_v2_r22, scenario_df, sber_rate=22, sa_rate=17.5)

Будем парсить по самым популярным срочностям:  

Прайм - 12  
Привилегия - 12  
Массовый - 12

### Сохранить сценарии

In [155]:
# для записи и чтения экселя
import pip
pip.main(['install', 'openpyxl'])

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://nxrm:8081/repository/pypi/simple


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [156]:
import openpyxl

In [157]:
# with pd.ExcelWriter("10_24_scenarious.xlsx") as writer:
#     parse_sc_v1_r20.to_excel(writer, sheet_name='parse_sc_v1_r20', index=True)
#     parse_sc_v1_r21.to_excel(writer, sheet_name='parse_sc_v1_r21', index=True)
#     parse_sc_v1_r22.to_excel(writer, sheet_name='parse_sc_v1_r22', index=True)
    
    
#     parse_sc_v2_r21.to_excel(writer, sheet_name='parse_sc_v2_r21', index=True)
#     parse_sc_v2_r22.to_excel(writer, sheet_name='parse_sc_v2_r22', index=True)

### Чтение сценариев

In [168]:
# для записи и чтения экселя
import pip
pip.main(['install', 'openpyxl'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://nxrm:8081/repository/pypi/simple


0

In [169]:
import openpyxl

In [172]:
parse_sc_v1_r20 = pd.read_excel("10_24_scenarious.xlsx", sheet_name='parse_sc_v1_r20', index_col='Unnamed: 0')

In [174]:
scenario_df = parse_sc_v1_r20.copy()

In [195]:
sc_names = ['parse_sc_v1_r20',
           'parse_sc_v1_r21',
           'parse_sc_v1_r22',
           'parse_sc_v2_r21',
           'parse_sc_v2_r22']

In [196]:
sc_dict = {}

for sc in sc_names:
    
    sc_dict[sc] = pd.read_excel("10_24_scenarious.xlsx", sheet_name=sc, index_col='Unnamed: 0')

### Чтение портфеля и моделей

In [175]:
port_folder = '../../data/portfolio_data'
portfolio = load_portfolio(train_end, port_folder)

In [176]:
# если хотим обучить модели - инициализировать спарк

from core.models.utils import run_spark_session
#spark = run_spark_session('check_calc')

# spark = None #если без обучения

In [177]:
folder = '../../data/trained_models'

sqlite_filepath = os.path.join(folder, 'modeldb_test.bin')

DB_URL = f"sqlite:///{sqlite_filepath}"
model_db = ModelDB(DB_URL)

In [178]:
# dir(model_db)

In [179]:
# (model_db.find_models()[34]).trained_models

In [180]:
# model_db.delete_trained_models()

In [181]:
# важная ячейка, необходимая для обучения моделей orbit

ENV_NAME = 'hmelevskoi_env'

os.environ['CC'] = 'x86_64-conda-linux-gnu-gcc'
os.environ['CXX'] = 'x86_64-conda-linux-gnu-g++'
os.environ['PATH'] = os.path.abspath(f'/tmp/envs/{ENV_NAME}/bin') + ':' + os.environ['PATH']

## Чтение таблицы

In [182]:
# path = 'prod_dadm_alm_sbx.almde_fl_dpst_current_accounts'

In [183]:
# table = spark.table(path)

In [184]:
# table.columns

### Старт расчетов

In [185]:
config: ForecastConfig = ForecastConfig(
    first_train_end_dt = train_end,
    horizon = horizon,
    trainers = DepositModels.trainers,
    data_loaders = DepositModels.dataloaders,
    calculator_type = DepositIterativeCalculator,
    calc_type = DepositsCalculationType,
    adapter_types = DepositModels.adapter_types,
    scenario_data = scenario_df,
    portfolio = portfolio
)
    
training_manager = TrainingManager(spark, config.trainers, folder, model_db)   
engine: ForecastEngine = ForecastEngine(spark, config, training_manager, overwrite_models=False)

In [186]:
%%time
engine.run_all()

missing models: []
add_models_from_bytes
plan_close_201402_202409 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
renewal_model_201401_202409 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
newbusiness_mass_model_buckets_201402_202409 - adapter <class 'core.models.newbusiness.buckets_structure.buckets_structure_mass_model.NewbusinessBucketsMassModelAdapter'>
newbusiness_priv_model_buckets_201402_202409 - adapter <class 'core.models.newbusiness.buckets_structure.buckets_structure_priv_model.NewbusinessBucketsPrivModelAdapter'>
newbusiness_vip_model_buckets_201402_202409 - adapter <class 'core.models.newbusiness.buckets_structure.buckets_structure_vip_model.NewbusinessBucketsVipModelAdapter'>
maturity_structure_mass_r0s0_201401_202409 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
maturity_structure_mass_r0s1_202001_202409 - adapter <class 'core.

In [187]:
# вывод
portfolio_res = engine.calc_results['Deposits']['portfolio'] 
agg_res = engine.calc_results['Deposits']['agg_data']
maturity = engine.calc_results['Deposits']['maturity']
CurrentAccounts = engine.calc_results['CurrentAccounts']
SavingAccounts = engine.calc_results['SavingAccounts']
# volumes = engine.calc_results['Volumes']

In [189]:
portfolio_res.groupby('report_dt').sum()['balance']

report_dt
2024-09-30    5.932406e+12
2024-10-31    5.952324e+12
2024-11-30    6.139631e+12
2024-12-31    6.298827e+12
Name: balance, dtype: float64

## Автоматизация расчетов

In [200]:
def make_forecast(scenario_df_name):
    
    
    scenario_df = sc_dict[scenario_df_name].copy()
    
    config: ForecastConfig = ForecastConfig(
    first_train_end_dt = train_end,
    horizon = horizon,
    trainers = DepositModels.trainers,
    data_loaders = DepositModels.dataloaders,
    calculator_type = DepositIterativeCalculator,
    calc_type = DepositsCalculationType,
    adapter_types = DepositModels.adapter_types,
    scenario_data = scenario_df,
    portfolio = portfolio
    )

    training_manager = TrainingManager(spark, config.trainers, folder, model_db)   
    engine: ForecastEngine = ForecastEngine(spark, config, training_manager, overwrite_models=False)
        
    engine.run_all()
    
    
    portfolio_res = engine.calc_results['Deposits']['portfolio'] 
    agg_res = engine.calc_results['Deposits']['agg_data']
    CurrentAccounts = engine.calc_results['CurrentAccounts']
    SavingAccounts = engine.calc_results['SavingAccounts']
    
    
    # записываем результаты прогноза
    with pd.ExcelWriter(f"./forecast_res/{scenario_df_name}.xlsx") as writer:
        portfolio_res.to_excel(writer, sheet_name='portfolio_res', index=False)
        agg_res.to_excel(writer, sheet_name='agg_res', index=False)
        CurrentAccounts.to_excel(writer, sheet_name='CurrentAccounts', index=False)
        SavingAccounts.to_excel(writer, sheet_name='SavingAccounts', index=False)
        pd.DataFrame(scenario_df).to_excel(writer, sheet_name='scenario', index=True)

In [201]:
for sc in sc_names:
    make_forecast(sc)

missing models: []
add_models_from_bytes
plan_close_201402_202409 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
renewal_model_201401_202409 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
newbusiness_mass_model_buckets_201402_202409 - adapter <class 'core.models.newbusiness.buckets_structure.buckets_structure_mass_model.NewbusinessBucketsMassModelAdapter'>
newbusiness_priv_model_buckets_201402_202409 - adapter <class 'core.models.newbusiness.buckets_structure.buckets_structure_priv_model.NewbusinessBucketsPrivModelAdapter'>
newbusiness_vip_model_buckets_201402_202409 - adapter <class 'core.models.newbusiness.buckets_structure.buckets_structure_vip_model.NewbusinessBucketsVipModelAdapter'>
maturity_structure_mass_r0s0_201401_202409 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
maturity_structure_mass_r0s1_202001_202409 - adapter <class 'core.

### Сохранение данных

In [19]:
# для записи и чтения экселя
# import pip
# pip.main(['install', 'openpyxl'])

In [20]:
# import openpyxl

In [21]:
# with pd.ExcelWriter("august_res_v2.xlsx") as writer:
#     portfolio_res.to_excel(writer, sheet_name='portfolio_res', index=False)
#     agg_res.to_excel(writer, sheet_name='agg_res', index=False)
#     maturity.to_excel(writer, sheet_name='maturity', index=False)
#     CurrentAccounts.to_excel(writer, sheet_name='CurrentAccounts', index=False)
#     SavingAccounts.to_excel(writer, sheet_name='SavingAccounts', index=False)
#     volumes.to_excel(writer, sheet_name='volumes', index=False)
#     pd.DataFrame(scenario_data).to_excel(writer, sheet_name='scenario', index=False)